In [1]:
import pickle
import torch
from utils.analysis_from_interaction import *
from egg.core.language_analysis import Disent
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim

# calculate metrics from stored interactions

In [2]:
granularities = ['granularity_coarse', 'granularity_fine']

In [3]:
datasets = ('(3,4)', '(3,8)', '(3,16)', '(4,4)', '(4,8)', '(5,4)')
n_attributes = (3, 3, 3, 4, 4, 5)
n_values = (4, 8, 16, 4, 8, 4)
n_epochs = 300
#paths = ['results/' + d + '_sample_scaling_10_balanced_False_vsf_3/' for d in datasets]
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]



In [4]:
### this cell is used to evaluate only the smallest dataset in a preliminary phase

datasets = ('(3,4)',)
n_attributes = (3,)
n_values = (4,)
n_epochs = 300
#paths = ['results/' + d + '_sample_scaling_10_balanced_False_vsf_3/' for d in datasets]
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]

In [5]:

datasets = ('(5,4)',)
n_attributes = (5,)
n_values = (4,)
n_epochs = 300
#paths = ['results/' + d + '_sample_scaling_10_balanced_False_vsf_3/' for d in datasets]
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]

In [4]:
context_unaware = False # whether original or context_unaware simulations are evaluated
length_cost = False # whether length_cost was applied
no_cost = True # set to True to consider baselines without a cost
if length_cost:
    if context_unaware and not no_cost:
        setting = 'length_cost/context_unaware'
    elif context_unaware and no_cost:
        setting = 'length_cost/no_cost_context_unaware'
    elif not context_unaware and not no_cost:
        setting = 'length_cost/context_aware'
    else:
        setting = 'length_cost/no_cost_context_aware'
else:
    if context_unaware:
        setting = 'context_unaware'
    else:
        setting = 'standard'

## entropy scores: MI, effectiveness, efficiency

In [ ]:
for d in range(len(datasets)):
    
    for run in range(5):

        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/' 
        print(path_to_run)
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = information_scores(interaction, attributes, values, normalizer="arithmetic")

        pickle.dump(scores, open(path_to_run + 'entropy_scores.pkl', 'wb'))
    

In [7]:
# this cell is to analyze results in standard with non-default granularity
for g in granularities:
    for d in range(len(datasets)):
        
        for run in range(5): 
            path_to_run = paths[d] + str(setting) + '/' + str(g) + '/' + str(run) + '/' 
            print(path_to_run)
            path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
            interaction = torch.load(path_to_interaction)

            attributes = n_attributes[d]
            values = n_values[d]
            scores = information_scores(interaction, attributes, values, normalizer="arithmetic")

            pickle.dump(scores, open(path_to_run + 'entropy_scores.pkl', 'wb'))
    

results/(5,4)_game_size_10_vsf_3/standard/granularity_coarse/0/


c:\Users\Colab\Documents\GitHub\emergent-abstractions\utils\analysis_from_interaction.py:312: RuntimeWarning: invalid value encountered in divide
  normalized_MI_context_dep = ((m_entropy_context_dep + c_entropy_context_dep - joint_entropy_context_dep)
c:\Users\Colab\Documents\GitHub\emergent-abstractions\utils\analysis_from_interaction.py:315: RuntimeWarning: invalid value encountered in divide
  (m_entropy_concept_x_context + c_entropy_concept_x_context - joint_entropy_concept_x_context)
c:\Users\Colab\Documents\GitHub\emergent-abstractions\utils\analysis_from_interaction.py:322: RuntimeWarning: invalid value encountered in divide
  normalized_effectiveness_context_dep = ((joint_entropy_context_dep - m_entropy_context_dep)
c:\Users\Colab\Documents\GitHub\emergent-abstractions\utils\analysis_from_interaction.py:324: RuntimeWarning: invalid value encountered in divide
  normalized_effectiveness_conc_x_cont = ((joint_entropy_concept_x_context - m_entropy_concept_x_context)
c:\Users\Cola

results/(5,4)_game_size_10_vsf_3/standard/granularity_coarse/1/
results/(5,4)_game_size_10_vsf_3/standard/granularity_coarse/2/
results/(5,4)_game_size_10_vsf_3/standard/granularity_coarse/3/
results/(5,4)_game_size_10_vsf_3/standard/granularity_coarse/4/


##  message length

In [ ]:
# we evaluated message length per hierarchy level after training but 
# you can also use the HierarchicalMessageLength callback and store the results 
# TODO: Message length results look weird, needs to be fixed!

for d in range(len(datasets)):
    
    for run in range(5): 
        
        path_to_run = paths[d] + '/' + str(setting) +'/' + str(run) + '/'
        path_to_interaction = (path_to_run + 'interactions/train/interactions_epoch' + str(n_epochs))
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = message_length_per_hierarchy_level(interaction, attributes)
        
        pickle.dump(scores, open(path_to_run + 'message_length_hierarchical.pkl', 'wb'))

##  symbol redundancy

In [ ]:
for d in range(len(datasets)):
    
    attributes = n_attributes[d]
    values = n_values[d]
    vs_factor = int(paths[d][-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1
    
    for run in range(5): 
        
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        #symbol_f = np.load(path_to_run + 'symbols_pernsame.npy')
        #path_to_interaction = (path_to_run + 'interactions/train/interactions_epoch' + str(n_epochs))
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)
        redundancy, MI = symbol_frequency(interaction, attributes, values, vocab_size)
        
        scores = {'symbol_redundancy': redundancy, 'MI_symbol-attribute_value': MI}
        
        pickle.dump(scores, open(path_to_run + 'symbol_redundancy.pkl', 'wb'))

In [6]:
### use this to evaluate experiments with non-default granularity

for g in granularities:
    for d in range(len(datasets)):
        
        attributes = n_attributes[d]
        values = n_values[d]
        vs_factor = int(paths[d][-2])
        vocab_size = (n_values[d] + 1) * vs_factor + 1
        
        for run in range(5): 
            
            path_to_run = paths[d] + str(setting) +'/' + str(g) + '/' + str(run) + '/'
            #symbol_f = np.load(path_to_run + 'symbols_pernsame.npy')
            #path_to_interaction = (path_to_run + 'interactions/train/interactions_epoch' + str(n_epochs))
            path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
            interaction = torch.load(path_to_interaction)
            redundancy, MI = symbol_frequency(interaction, attributes, values, vocab_size)
            
            scores = {'symbol_redundancy': redundancy, 'MI_symbol-attribute_value': MI}
            
            pickle.dump(scores, open(path_to_run + 'symbol_redundancy.pkl', 'wb'))

c:\Users\Colab\Documents\GitHub\emergent-abstractions\utils\analysis_from_interaction.py:446: RuntimeWarning: invalid value encountered in divide
  return symbol_frequency / att_val_frequency, mutual_information
c:\Users\Colab\Documents\GitHub\emergent-abstractions\utils\analysis_from_interaction.py:446: RuntimeWarning: invalid value encountered in divide
  return symbol_frequency / att_val_frequency, mutual_information
c:\Users\Colab\Documents\GitHub\emergent-abstractions\utils\analysis_from_interaction.py:446: RuntimeWarning: invalid value encountered in divide
  return symbol_frequency / att_val_frequency, mutual_information
c:\Users\Colab\Documents\GitHub\emergent-abstractions\utils\analysis_from_interaction.py:446: RuntimeWarning: invalid value encountered in divide
  return symbol_frequency / att_val_frequency, mutual_information
c:\Users\Colab\Documents\GitHub\emergent-abstractions\utils\analysis_from_interaction.py:446: RuntimeWarning: invalid value encountered in divide
  retu

##  compositionality scores: topsim, posdis, bosdis

### topsim

In [ ]:
# topsim
# although topsim values are stored throughout training if callbacks are verbose, we reevaluate the
# final topsim scores with more data points 

samples = 5000 # maybe shuffle from these because otherwise I just take the first 5,000 (which might not be the best)
for d, dataset in enumerate(datasets):
    
    dim = [n_values[d]]*n_attributes[d]
    
    for run in range(5):
        print("dataset", dataset, "run", run)
        
        topsim_final = {}
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        
        TOPSIM = TopographicSimilarityConceptLevel(dim, is_gumbel=True)
        
        for mode in ['train', 'val']:

            if mode == 'train':
                interaction = torch.load(path_to_interaction_train)
            elif mode == 'val':
                interaction = torch.load(path_to_interaction_val)
                
                  
            messages = interaction.message.argmax(dim=-1)
            sender_input = interaction.sender_input
            n_targets = int(sender_input.shape[1]/2)
            # get target objects and fixed vectors to re-construct concepts
            target_objects = sender_input[:, :n_targets]
            target_objects = k_hot_to_attributes(target_objects, n_values[d])
            # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
            (objects, fixed) = retrieve_concepts_sampling(target_objects)
            # add one such that zero becomes an empty attribute for the calculation (_)
            objects = objects + 1
            concepts = torch.from_numpy(objects * (np.array(fixed)))
            specific_idx = np.where(np.sum(fixed, axis=1)==n_attributes[d])[0]
            messages_specific = messages[specific_idx]
            concepts_specific = concepts[specific_idx]
            
            generic_idx = np.where(np.sum(fixed, axis=1)==1)[0]
            messages_generic = messages[generic_idx]
            concepts_generic = concepts[generic_idx]

            messages = [msg.tolist() for msg in messages]
            messages_specific = [msg.tolist() for msg in messages_specific]
            messages_generic = [msg.tolist() for msg in messages_generic]

            encoded_input = encode_target_concepts_for_topsim(sender_input)
            # randomly take samples when more than 5000 samples are available
            # if len(encoded_input) > samples: 
            #     print("sampling")
            #     sample_indices = random.sample(range(len(encoded_input)), samples)
            #     sampled_input = [encoded_input[i] for i in sample_indices]
            #     sampled_messages = [messages[i] for i in sample_indices]
            #     print("start computing")
            #     print(len(sampled_input), len(sampled_input[0]), len(sampled_input[0][0]))
            #     topsim = TOPSIM.compute_topsim(sampled_input, sampled_messages)
            # else:
            topsim = TOPSIM.compute_topsim(encoded_input[0:samples], messages[0:samples]) # default: hausdorff distance for concepts, edit distance for messages
            # if len(concepts_specific) > samples:
            #     print("sampling specific")
            #     sample_indices_specific = random.sample(range(len(concepts_specific)), samples)
            #     sampled_input_specific = [concepts_specific[i] for i in sample_indices_specific]
            #     sampled_messages_specific = [messages_specific[i] for i in sample_indices_specific]
            #     topsim_specific = TOPSIM.compute_topsim(sampled_input_specific, sampled_messages_specific, 
            #                                             meaning_distance_fn="edit")
            # else:
            topsim_specific = TOPSIM.compute_topsim(concepts_specific[0:samples], messages_specific[0:samples], 
                                                        meaning_distance_fn="edit")
            
            topsim_generic = TOPSIM.compute_topsim(concepts_generic[0:samples], messages_generic[0:samples],
                                                   meaning_distance_fn="edit")

            print('... topsim computed')

            topsim_final['topsim_' + mode] = topsim
            topsim_final['topsim_specific_' + mode] = topsim_specific
            topsim_final['topsim_generic_' + mode] = topsim_generic
    
        pickle.dump(topsim_final, open(path_to_run +  "topsim_final.pkl", "wb" ) )
        print(topsim_final)        

In [7]:
###use this for non-default granularities

# topsim
# although topsim values are stored throughout training if callbacks are verbose, we reevaluate the
# final topsim scores with more data points 

samples = 5000 # maybe shuffle from these because otherwise I just take the first 5,000 (which might not be the best)

for g in granularities:
    for d, dataset in enumerate(datasets):
        
        dim = [n_values[d]]*n_attributes[d]
        
        for run in range(5):
            print("dataset", dataset, "granularity", g, "run", run)
            
            topsim_final = {}
            path_to_run = paths[d] + str(setting) +'/' +str(g) + '/' + str(run) + '/'
            path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
            path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
            
            TOPSIM = TopographicSimilarityConceptLevel(dim, is_gumbel=True)
            
            for mode in ['train', 'val']:

                if mode == 'train':
                    interaction = torch.load(path_to_interaction_train)
                elif mode == 'val':
                    interaction = torch.load(path_to_interaction_val)
                    
                    
                messages = interaction.message.argmax(dim=-1)
                sender_input = interaction.sender_input
                n_targets = int(sender_input.shape[1]/2)
                # get target objects and fixed vectors to re-construct concepts
                target_objects = sender_input[:, :n_targets]
                target_objects = k_hot_to_attributes(target_objects, n_values[d])
                # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
                (objects, fixed) = retrieve_concepts_sampling(target_objects)
                # add one such that zero becomes an empty attribute for the calculation (_)
                objects = objects + 1
                concepts = torch.from_numpy(objects * (np.array(fixed)))
                specific_idx = np.where(np.sum(fixed, axis=1)==n_attributes[d])[0]
                messages_specific = messages[specific_idx]
                concepts_specific = concepts[specific_idx]
                
                generic_idx = np.where(np.sum(fixed, axis=1)==1)[0]
                messages_generic = messages[generic_idx]
                concepts_generic = concepts[generic_idx]

                messages = [msg.tolist() for msg in messages]
                messages_specific = [msg.tolist() for msg in messages_specific]
                messages_generic = [msg.tolist() for msg in messages_generic]

                encoded_input = encode_target_concepts_for_topsim(sender_input)
                # randomly take samples when more than 5000 samples are available
                # if len(encoded_input) > samples: 
                #     print("sampling")
                #     sample_indices = random.sample(range(len(encoded_input)), samples)
                #     sampled_input = [encoded_input[i] for i in sample_indices]
                #     sampled_messages = [messages[i] for i in sample_indices]
                #     print("start computing")
                #     print(len(sampled_input), len(sampled_input[0]), len(sampled_input[0][0]))
                #     topsim = TOPSIM.compute_topsim(sampled_input, sampled_messages)
                # else:
                topsim = TOPSIM.compute_topsim(encoded_input[0:samples], messages[0:samples]) # default: hausdorff distance for concepts, edit distance for messages
                # if len(concepts_specific) > samples:
                #     print("sampling specific")
                #     sample_indices_specific = random.sample(range(len(concepts_specific)), samples)
                #     sampled_input_specific = [concepts_specific[i] for i in sample_indices_specific]
                #     sampled_messages_specific = [messages_specific[i] for i in sample_indices_specific]
                #     topsim_specific = TOPSIM.compute_topsim(sampled_input_specific, sampled_messages_specific, 
                #                                             meaning_distance_fn="edit")
                # else:
                topsim_specific = TOPSIM.compute_topsim(concepts_specific[0:samples], messages_specific[0:samples], 
                                                            meaning_distance_fn="edit")
                
                topsim_generic = TOPSIM.compute_topsim(concepts_generic[0:samples], messages_generic[0:samples],
                                                    meaning_distance_fn="edit")

                print('... topsim computed')

                topsim_final['topsim_' + mode] = topsim
                topsim_final['topsim_specific_' + mode] = topsim_specific
                topsim_final['topsim_generic_' + mode] = topsim_generic
        
            pickle.dump(topsim_final, open(path_to_run +  "topsim_final.pkl", "wb" ) )
            print(topsim_final)        

dataset (3,4) granularity granularity_coarse run 0
... topsim computed
... topsim computed
{'topsim_train': 0.3845364887873862, 'topsim_specific_train': 0.466795721631349, 'topsim_generic_train': 0.6368295414776609, 'topsim_val': 0.3942719764187433, 'topsim_specific_val': 0.4607090763085089, 'topsim_generic_val': 0.6930537137443035}
dataset (3,4) granularity granularity_coarse run 1
... topsim computed
... topsim computed
{'topsim_train': 0.40042203610422955, 'topsim_specific_train': 0.49598938931395503, 'topsim_generic_train': 0.3947235793890288, 'topsim_val': 0.4098081740291837, 'topsim_specific_val': 0.5060706191247804, 'topsim_generic_val': 0.5608943751287022}
dataset (3,4) granularity granularity_coarse run 2
... topsim computed
... topsim computed
{'topsim_train': 0.39764658795310004, 'topsim_specific_train': 0.5312698397372431, 'topsim_generic_train': 0.5708159309876886, 'topsim_val': 0.39860591941413565, 'topsim_specific_val': 0.5239743543798365, 'topsim_generic_val': 0.6776226

KeyboardInterrupt: 

#### Topsim over time

In [ ]:
for d, dataset in enumerate(datasets):
    
    for run in range(5):
        print("dataset", dataset, "run", run)
        
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        
        for mode in ['train', 'val']:

            if mode == 'train':
                interaction = torch.load(path_to_interaction_train)
            elif mode == 'val':
                interaction = torch.load(path_to_interaction_val)

        messages = interaction.message.argmax(dim=-1)
        sender_input = interaction.sender_input
        messages = [msg.tolist() for msg in messages]
        encoded_input = encode_target_concepts_for_topsim(sender_input)
        dim = [n_values[0]] * n_attributes[0]
        TOPSIM = TopographicSimilarityConceptLevel(dim, is_gumbel=True)
        
        samples = 5000
        num_batches = len(messages) // samples + (len(messages) % samples > 0)
        topsim_over_time = []
        
        for i in range(num_batches):
            messages_batch = messages[i * samples:(i + 1) * samples]
            topsim = TOPSIM.compute_topsim(encoded_input[i * samples:(i + 1) * samples], messages_batch)
            topsim_over_time.append(topsim)
            
        pickle.dump(topsim_over_time, open(path_to_run +  "topsim_over_time.pkl", "wb" ) )

In [ ]:
### use this for non-default granularities

for g in granularities:

    for d, dataset in enumerate(datasets):
        
        for run in range(5):
            print("dataset", dataset, "granularity", g, "run", run)
            
            path_to_run = paths[d] + str(setting) +'/' + str(g)+ '/' + str(run) + '/'
            path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
            path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
            
            for mode in ['train', 'val']:

                if mode == 'train':
                    interaction = torch.load(path_to_interaction_train)
                elif mode == 'val':
                    interaction = torch.load(path_to_interaction_val)

            messages = interaction.message.argmax(dim=-1)
            sender_input = interaction.sender_input
            messages = [msg.tolist() for msg in messages]
            encoded_input = encode_target_concepts_for_topsim(sender_input)
            dim = [n_values[0]] * n_attributes[0]
            TOPSIM = TopographicSimilarityConceptLevel(dim, is_gumbel=True)
            
            samples = 5000
            num_batches = len(messages) // samples + (len(messages) % samples > 0)
            topsim_over_time = []
            
            for i in range(num_batches):
                messages_batch = messages[i * samples:(i + 1) * samples]
                topsim = TOPSIM.compute_topsim(encoded_input[i * samples:(i + 1) * samples], messages_batch)
                topsim_over_time.append(topsim)
                
            pickle.dump(topsim_over_time, open(path_to_run +  "topsim_over_time.pkl", "wb" ) )

dataset (3,4) granularity granularity_coarse run 0
dataset (3,4) granularity granularity_coarse run 1
dataset (3,4) granularity granularity_coarse run 2
dataset (3,4) granularity granularity_coarse run 3
dataset (3,4) granularity granularity_coarse run 4
dataset (3,4) granularity granularity_fine run 0
dataset (3,4) granularity granularity_fine run 1
dataset (3,4) granularity granularity_fine run 2
dataset (3,4) granularity granularity_fine run 3
dataset (3,4) granularity granularity_fine run 4


### Posdis and Bosdis

In [ ]:
# use Disent callback from egg

for d in range(len(datasets)): 
    
    path = paths[d]
    dim = [n_values[d]] * n_attributes[d]
    n_features = n_attributes[d] * n_values[d]
    vs_factor = int(path[-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1
    
    print("data set", dim)
    
    for run in range(5):
        
        posdis_bosdis = {}
    
        path_to_run = paths[d] + '/' + str(setting) +'/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)
        
        messages = interaction.message.argmax(dim=-1)
        sender_input = interaction.sender_input
        n_targets = int(sender_input.shape[1]/2)
        # get target objects and fixed vectors to re-construct concepts
        target_objects = sender_input[:, :n_targets]
        target_objects = k_hot_to_attributes(target_objects, n_values[d])
        # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
        (objects, fixed) = retrieve_concepts_sampling(target_objects)
        # add one such that zero becomes an empty attribute for the calculation (_)
        objects = objects + 1
        concepts = torch.from_numpy(objects * (np.array(fixed)))

        # concrete/specific concepts: where all attributes are fixed
        concepts_specific = torch.tensor(
            objects[torch.sum(torch.from_numpy(fixed), dim=1) == n_attributes[d]])
        messages_specific = messages[torch.sum(torch.from_numpy(fixed), dim=1) == n_attributes[d]]

        # generic concepts: where only one attribute is fixed
        concepts_generic = torch.tensor(
            objects[torch.sum(torch.from_numpy(fixed), dim=1) == 1])
        messages_generic = messages[torch.sum(torch.from_numpy(fixed), dim=1) == 1]
        
        posdis_specific = Disent.posdis(concepts_specific, messages_specific)
        bosdis_specific = Disent.bosdis(concepts_specific, messages_specific, vocab_size)

        posdis_generic = Disent.posdis(concepts_generic, messages_generic)
        bosdis_generic = Disent.bosdis(concepts_generic, messages_generic, vocab_size)
        
        posdis = Disent.posdis(torch.from_numpy(objects), messages)
        bosdis = Disent.bosdis(torch.from_numpy(objects), messages, vocab_size)
        
        posdis_bosdis['posdis_specific'] = posdis_specific
        posdis_bosdis['bosdis_specific'] = bosdis_specific
        posdis_bosdis['posdis_generic'] = posdis_generic
        posdis_bosdis['bosdis_generic'] = bosdis_generic
        posdis_bosdis['posdis'] = posdis
        posdis_bosdis['bosdis'] = bosdis

        print(posdis_bosdis)
    
        pickle.dump(posdis_bosdis, open(path_to_run + "posdis_bosdis.pkl", "wb" ) )

In [ ]:
### use this for non-default granularities

# use Disent callback from egg
for g in granularities:

    for d in range(len(datasets)): 
        
        path = paths[d]
        dim = [n_values[d]] * n_attributes[d]
        n_features = n_attributes[d] * n_values[d]
        vs_factor = int(path[-2])
        vocab_size = (n_values[d] + 1) * vs_factor + 1
        
        print("data set", dim, str(g))

        
        for run in range(5):
            
            posdis_bosdis = {}
        
            path_to_run = paths[d] + '/' + str(setting) +'/' + str(g) + '/' + str(run) + '/'
            path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
            interaction = torch.load(path_to_interaction_train)
            
            messages = interaction.message.argmax(dim=-1)
            sender_input = interaction.sender_input
            n_targets = int(sender_input.shape[1]/2)
            # get target objects and fixed vectors to re-construct concepts
            target_objects = sender_input[:, :n_targets]
            target_objects = k_hot_to_attributes(target_objects, n_values[d])
            # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
            (objects, fixed) = retrieve_concepts_sampling(target_objects)
            # add one such that zero becomes an empty attribute for the calculation (_)
            objects = objects + 1
            concepts = torch.from_numpy(objects * (np.array(fixed)))

            # concrete/specific concepts: where all attributes are fixed
            concepts_specific = torch.tensor(
                objects[torch.sum(torch.from_numpy(fixed), dim=1) == n_attributes[d]])
            messages_specific = messages[torch.sum(torch.from_numpy(fixed), dim=1) == n_attributes[d]]

            # generic concepts: where only one attribute is fixed
            concepts_generic = torch.tensor(
                objects[torch.sum(torch.from_numpy(fixed), dim=1) == 1])
            messages_generic = messages[torch.sum(torch.from_numpy(fixed), dim=1) == 1]
            
            posdis_specific = Disent.posdis(concepts_specific, messages_specific)
            bosdis_specific = Disent.bosdis(concepts_specific, messages_specific, vocab_size)

            posdis_generic = Disent.posdis(concepts_generic, messages_generic)
            bosdis_generic = Disent.bosdis(concepts_generic, messages_generic, vocab_size)
            
            posdis = Disent.posdis(torch.from_numpy(objects), messages)
            bosdis = Disent.bosdis(torch.from_numpy(objects), messages, vocab_size)
            
            posdis_bosdis['posdis_specific'] = posdis_specific
            posdis_bosdis['bosdis_specific'] = bosdis_specific
            posdis_bosdis['posdis_generic'] = posdis_generic
            posdis_bosdis['bosdis_generic'] = bosdis_generic
            posdis_bosdis['posdis'] = posdis
            posdis_bosdis['bosdis'] = bosdis

            print(posdis_bosdis)
        
            pickle.dump(posdis_bosdis, open(path_to_run + "posdis_bosdis.pkl", "wb" ) )

data set [4, 4, 4] granularity_coarse
{'posdis_specific': 0.08977968245744705, 'bosdis_specific': 0.2893837094306946, 'posdis_generic': 0.014423076994717121, 'bosdis_generic': 0.40043753385543823, 'posdis': 0.06159316375851631, 'bosdis': 0.34295418858528137}
{'posdis_specific': 0.5445482134819031, 'bosdis_specific': 0.30714866518974304, 'posdis_generic': 0.08193547278642654, 'bosdis_generic': 0.24916066229343414, 'posdis': 0.3510983884334564, 'bosdis': 0.28555911779403687}
{'posdis_specific': 0.8766304850578308, 'bosdis_specific': 0.7075303792953491, 'posdis_generic': 0.07515589892864227, 'bosdis_generic': 0.3101643919944763, 'posdis': 0.44335219264030457, 'bosdis': 0.3930688798427582}
{'posdis_specific': 0.4346048831939697, 'bosdis_specific': 0.3561984598636627, 'posdis_generic': 0.04214636981487274, 'bosdis_generic': 0.32516175508499146, 'posdis': 0.2392079383134842, 'bosdis': 0.35278433561325073}
{'posdis_specific': 0.5778630375862122, 'bosdis_specific': 0.30237147212028503, 'posdis

#### Posdis and bosdis concept x context

In [ ]:
# posdis concept x context
for d in range(len(datasets)):

    vs_factor = int(paths[d][-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1

    for run in range(5):
        path_to_run = paths[d] + '/' + str(setting) + '/' + str(run) + '/'
        print(path_to_run)
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        #print(path_to_interaction)
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = posdis(interaction, attributes, values, vocab_size)

        pickle.dump(scores, open(path_to_run + 'posdis_scores.pkl', 'wb'))

In [ ]:
### use this for non-default granularities
##there is a mistake here

# posdis concept x context
for g in granularities:
    for d in range(len(datasets)):

        vs_factor = int(paths[d][-2])
        vocab_size = (n_values[d] + 1) * vs_factor + 1

        for run in range(5):
            path_to_run = paths[d] + '/' + str(setting) + '/' + str(g) + '/' + str(run) + '/'
            print(path_to_run)
            path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
            #print(path_to_interaction)
            interaction = torch.load(path_to_interaction)

            attributes = n_attributes[d]
            values = n_values[d]
            scores = posdis(interaction, attributes, values, vocab_size)

            pickle.dump(scores, open(path_to_run + 'posdis_scores.pkl', 'wb'))

results/(3,4)_game_size_10_vsf_3//standard/granularity_coarse/0/


TypeError: 'float' object is not callable

## co-occurrences

In [ ]:
# Not yet implemented:

for d in range(len(datasets)):
    
    vs_factor = int(paths[d][-2])
    
    for run in range(5): 
        
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        
        scores = cooccurrence_per_hierarchy_level(interaction, attributes, values, vs_factor)

        print(scores)
        
        pickle.dump(scores, open(path_to_run + 'normalized_cooccurrence.pkl', 'wb'))
    